In [4]:
from app import dimension_properties, indicator_properties

In [5]:
indicator_properties

,Indicator,Description,Unit,Dimension,Impact,minYear,maxYear,Category
0,GE3,"Ratio non-CO2 emissions (CH4, N2O and F-gas) i...",Tons CO2e per Capita,Natural Capital Protection,negative,1961,2020,GE
1,SE1,Palma Ratio,Ratio,Social Inclusion,negative,1967,2020,SE
2,ME1,Total domestic material consumption (DMC) per ...,kg per constant 2005 GDP,Efficient and Sustainable Resource Use,negative,1970,2020,ME
3,GB3,"Getting paid, laws and regulations for equal g...",Score (0-100),Social Inclusion,positive,1971,2020,GB
4,GN1,Share of patent publications in environmental ...,Percent,Green Economic Opportunities,positive,1980,2020,GN
5,CV3,Share of terrestrial and marine protected area...,Percent,Natural Capital Protection,positive,1990,2020,CV
6,EE2,Renewable energy share in the total final ener...,Percent,Efficient and Sustainable Resource Use,positive,1990,2020,EE
7,GE2,"Ratio non-CO2 emissions (CH4, N2O and F-gas) e...",Tons CO2e per Capita,Natural Capital Protection,negative,1990,2020,GE
8,EE1,Energy intensity level of primary energy,Megajoules per constant 2011 purchasing power ...,Efficient and Sustainable Resource Use,negative,1990,2020,EE
9,BE2,Forest area as a proportion of total land area,Percent,Natural Capital Protection,positive,1990,2020,BE


In [1]:
import pandas as pd
from app import indicator_data, indicator_properties, data
from pages.simulation import scenario_data_dictionnary
import plotly.express as px

In [2]:
ISO = 'FRA'
data.query("ISO == @ISO")[['ISO', 'Variable', 'Aggregation', 'Year', 'Value']]

,ISO,Variable,Aggregation,Year,Value
102676,FRA,Index,Index,2005.0,65.10
102677,FRA,AB1,Indicator,2005.0,93.10
102678,FRA,AB2,Indicator,2005.0,97.50
102679,FRA,AB3,Indicator,2005.0,47.09
102680,FRA,BE1,Indicator,2005.0,59.73
...,...,...,...,...,...
104066,FRA,SP,Category,2019.0,85.42
104067,FRA,SI,Dimension,2019.0,89.31
104068,FRA,NCP,Dimension,2019.0,79.61
104069,FRA,ESRU,Dimension,2019.0,68.41


In [11]:
data

,Variable,ISO,Value,Aggregation,Year,Country,Continent,UNregion,IncomeLevel,Region,Variable_name,Continental_Rank,Income_Rank,Category,Dimension
0,AB1,AFG,NaN,Indicator,2005.0,Afghanistan,Asia,Southern Asia,Low income,South Asia,NaN,NaN,NaN,NaN,NaN
1,AB2,AFG,18.50,Indicator,2005.0,Afghanistan,Asia,Southern Asia,Low income,South Asia,NaN,62.0,11.0,NaN,NaN
2,AB3,AFG,2.34,Indicator,2005.0,Afghanistan,Asia,Southern Asia,Low income,South Asia,NaN,85.0,33.0,NaN,NaN
3,BE1,AFG,0.00,Indicator,2005.0,Afghanistan,Asia,Southern Asia,Low income,South Asia,NaN,86.0,50.0,NaN,NaN
4,BE2,AFG,2.07,Indicator,2005.0,Afghanistan,Asia,Southern Asia,Low income,South Asia,NaN,64.0,40.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337117,SP,ZWE,41.40,Category,2019.0,Zimbabwe,Africa,Eastern Africa,Lower middle income,Sub-Saharan Africa,Social protection,12.0,24.0,SP,Social Inclusion
337118,SI,ZWE,41.71,Dimension,2019.0,Zimbabwe,Africa,Eastern Africa,Lower middle income,Sub-Saharan Africa,Social inclusion,24.0,37.0,NaN,NaN
337119,NCP,ZWE,78.42,Dimension,2019.0,Zimbabwe,Africa,Eastern Africa,Lower middle income,Sub-Saharan Africa,Natural capital protection,1.0,1.0,NaN,NaN
337120,ESRU,ZWE,57.58,Dimension,2019.0,Zimbabwe,Africa,Eastern Africa,Lower middle income,Sub-Saharan Africa,Efficient and sustainable resource use,29.0,25.0,NaN,NaN


In [4]:
def category_heatmap(ISO):
    REF_1 = 'AVG_' + "_".join(data[data.ISO == ISO][["IncomeLevel"]
                                                    ].drop_duplicates().values[0].tolist())
    REF_2 = 'AVG_' + "_".join(data[data.ISO == ISO][["Continent"]
                                                    ].drop_duplicates().values[0].tolist())

    df = data[(data.ISO.isin([ISO, REF_1, REF_2])) &
              (data.Aggregation == 'Index')].fillna(0)
    
    return df

In [6]:
category_heatmap('FRA')

,Variable,ISO,Value,Aggregation,Year,Country,Continent,UNregion,IncomeLevel,Region,Variable_name,Continental_Rank,Income_Rank,Category,Dimension
102676,Index,FRA,65.10,Index,2005.0,France,Europe,Western Europe,High income,Europe & Central Asia,0,12.0,12.0,0,0
102769,Index,FRA,65.66,Index,2006.0,France,Europe,Western Europe,High income,Europe & Central Asia,0,12.0,13.0,0,0
102862,Index,FRA,65.64,Index,2007.0,France,Europe,Western Europe,High income,Europe & Central Asia,0,14.0,14.0,0,0
102955,Index,FRA,66.74,Index,2008.0,France,Europe,Western Europe,High income,Europe & Central Asia,0,10.0,10.0,0,0
103048,Index,FRA,66.46,Index,2009.0,France,Europe,Western Europe,High income,Europe & Central Asia,0,10.0,10.0,0,0
103141,Index,FRA,66.83,Index,2010.0,France,Europe,Western Europe,High income,Europe & Central Asia,0,12.0,12.0,0,0
103234,Index,FRA,67.76,Index,2011.0,France,Europe,Western Europe,High income,Europe & Central Asia,0,12.0,12.0,0,0
103327,Index,FRA,68.00,Index,2012.0,France,Europe,Western Europe,High income,Europe & Central Asia,0,12.0,12.0,0,0
103420,Index,FRA,67.93,Index,2013.0,France,Europe,Western Europe,High income,Europe & Central Asia,0,13.0,13.0,0,0
103513,Index,FRA,68.22,Index,2014.0,France,Europe,Western Europe,High income,Europe & Central Asia,0,13.0,13.0,0,0


In [ ]:
    REF_1 = 'AVG_' + "_".join(data[data.ISO == ISO][["IncomeLevel"]
                                                    ].drop_duplicates().values[0].tolist())
    REF_2 = 'AVG_' + "_".join(data[data.ISO == ISO][["Continent"]
                                                    ].drop_duplicates().values[0].tolist())

In [1]:
0.8 * 3.9

3.12

In [3]:
data.loc['']

,Variable,ISO,Value,Aggregation,Year,Country,Continent,UNregion,IncomeLevel,Region,Variable_name,Continental_Rank,Income_Rank,Category,Dimension
0,AB1,AFG,NaN,Indicator,2005.0,Afghanistan,Asia,Southern Asia,Low income,South Asia,NaN,NaN,NaN,NaN,NaN
1,AB2,AFG,18.50,Indicator,2005.0,Afghanistan,Asia,Southern Asia,Low income,South Asia,NaN,62.0,11.0,NaN,NaN
2,AB3,AFG,2.34,Indicator,2005.0,Afghanistan,Asia,Southern Asia,Low income,South Asia,NaN,85.0,33.0,NaN,NaN
3,BE1,AFG,0.00,Indicator,2005.0,Afghanistan,Asia,Southern Asia,Low income,South Asia,NaN,86.0,50.0,NaN,NaN
4,BE2,AFG,2.07,Indicator,2005.0,Afghanistan,Asia,Southern Asia,Low income,South Asia,NaN,64.0,40.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337117,SP,ZWE,41.40,Category,2019.0,Zimbabwe,Africa,Eastern Africa,Lower middle income,Sub-Saharan Africa,Social protection,12.0,24.0,SP,Social Inclusion
337118,SI,ZWE,41.71,Dimension,2019.0,Zimbabwe,Africa,Eastern Africa,Lower middle income,Sub-Saharan Africa,Social inclusion,24.0,37.0,NaN,NaN
337119,NCP,ZWE,78.42,Dimension,2019.0,Zimbabwe,Africa,Eastern Africa,Lower middle income,Sub-Saharan Africa,Natural capital protection,1.0,1.0,NaN,NaN
337120,ESRU,ZWE,57.58,Dimension,2019.0,Zimbabwe,Africa,Eastern Africa,Lower middle income,Sub-Saharan Africa,Efficient and sustainable resource use,29.0,25.0,NaN,NaN


In [155]:
import plotly.express as px


def missing_bar_plot(ISO):
    cats = ['EE', 'EW', 'SL', 'ME',
            'EQ', 'GE', 'BE', 'CV',
            'AB', 'GB', 'SE', 'SP',
            'GV', 'GT', 'GJ', 'GN']
    

    plot_df = (
        missing_data.loc[ISO].reset_index().merge(indicator_properties[['Category', "Dimension"]].drop_duplicates(), on='Category')
                    .merge(data[['Variable_name', 'Variable']].drop_duplicates(), left_on='Category', right_on='Variable')
    )
    plot_df_bis = plot_df.copy().assign(Dimension='Missing')
    plot_df_bis['Data availability (%)'] = (100 - plot_df_bis['Data availability (%)']).round(2)

    plot_df = pd.concat([plot_df, plot_df_bis])
    fig = px.bar(plot_df,
             y='Category',
             x='Data availability (%)',
             color='Dimension',
             barmode='stack',
             orientation='h',
            text='Data availability (%)',
             hover_data={'Variable_name': True, 'Dimension': False, 'Data availability (%)': False},
            labels={'Variable_name': 'Category'},
            color_discrete_map={
                           "Social Inclusion": "#d9b5c9",
                           "Natural Capital Protection": "#f7be49",
                           "Efficient and Sustainable Resource Use": "#8fd1e7",
                           "Green Economic Opportunities": "#9dcc93",
                           "Missing": '#D3D3D3'
                       },
                ).update_layout(height=1000,
                                plot_bgcolor='rgba(0, 0, 0, 0)',
                                legend=dict(
        yanchor="top",
        y=-0.05,
        xanchor="center",
        x=0,
        title=''
        
    ),
    ).update_traces(texttemplate='%{text:.2s}%', textposition='inside', textfont=dict(
        family="sans serif",
        size=18,
        color="white"
    )).update_yaxes(title="").update_xaxes(showticklabels=False)

    return fig


In [156]:
missing_bar_plot('FRA')